# バランスデータセット学習
このノートブックでは、正例と負例が1:1になるようにデータセットをバランス調整して学習します。

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    roc_auc_score, 
    f1_score, 
    accuracy_score,
    confusion_matrix,
    classification_report
)
import optuna
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')

# 再現性のためのシード設定
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("ライブラリのインポート完了")

In [ ]:
# データ読み込み
train_df = pd.read_csv("/home/user/bank/data/train.csv")
test_df = pd.read_csv("/home/user/bank/data/test.csv")

print(f"Train shape: {train_df.shape}")
print(f"Test shape: {test_df.shape}")
print(f"\nTarget distribution (元データ):")
print(train_df['y'].value_counts())
print(f"\nPositive rate: {train_df['y'].mean():.4f}")

## バランスデータセットの作成
正例と負例が1:1になるように、アンダーサンプリングを実施します。

In [ ]:
def create_balanced_dataset(df, target_col='y', random_state=42):
    """
    正例と負例が1:1になるようにバランス調整したデータセットを作成
    
    Parameters:
    -----------
    df : DataFrame
        元のデータフレーム
    target_col : str
        ターゲット列の名前
    random_state : int
        ランダムシード
    
    Returns:
    --------
    balanced_df : DataFrame
        バランス調整済みのデータフレーム
    """
    # 正例と負例に分離
    positive_samples = df[df[target_col] == 1]
    negative_samples = df[df[target_col] == 0]
    
    print(f"正例数: {len(positive_samples)}")
    print(f"負例数: {len(negative_samples)}")
    
    # 少数クラスの数を基準にする
    min_samples = min(len(positive_samples), len(negative_samples))
    
    # 各クラスから同じ数だけランダムサンプリング
    positive_balanced = positive_samples.sample(n=min_samples, random_state=random_state)
    negative_balanced = negative_samples.sample(n=min_samples, random_state=random_state)
    
    # 結合してシャッフル
    balanced_df = pd.concat([positive_balanced, negative_balanced], axis=0)
    balanced_df = balanced_df.sample(frac=1, random_state=random_state).reset_index(drop=True)
    
    print(f"\nバランス調整後のデータセットサイズ: {len(balanced_df)}")
    print(f"正例数: {(balanced_df[target_col] == 1).sum()}")
    print(f"負例数: {(balanced_df[target_col] == 0).sum()}")
    print(f"正例率: {balanced_df[target_col].mean():.4f}")
    
    return balanced_df

# バランスデータセットを作成
train_balanced = create_balanced_dataset(train_df, target_col='y', random_state=RANDOM_STATE)

## 特徴量エンジニアリング

In [ ]:
def feature_engineering(df, is_train=True):
    """
    特徴量エンジニアリング関数（ワンホットエンコーディング版）
    """
    df = df.copy()
    
    # ===== 1. 数値特徴量の変換 =====
    df['age_group'] = pd.cut(df['age'], bins=16).astype(str)
    # LightGBMはJSON特殊文字をサポートしないため、すべて置換
    df['age_group'] = df['age_group'].str.replace(r'[\(\)\[\]\{\}\:\"\,\.\s]', '_', regex=True)
    
    # balance の対数変換
    df['balance_log'] = np.log1p(df['balance'] - df['balance'].min() + 1)
    df['balance_positive'] = (df['balance'] > 0).astype(int)
    df['balance_negative'] = (df['balance'] < 0).astype(int)
    
    # ===== 2. 時系列特徴量 =====
    df['duration_per_day'] = df['duration'] / (df['day'] + 1)
    df['campaign_efficiency'] = df['duration'] / (df['campaign'] + 1)
    df['duration_log'] = np.log1p(df['duration'])
    
    # previous関連
    df['has_previous_contact'] = (df['pdays'] != -1).astype(int)
    df['previous_per_pdays'] = df['previous'] / (df['pdays'].replace(-1, 1) + 1)
    
    # ===== 3. 月のマッピングと周期性エンコーディング =====
    month_mapping = {
        'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4,
        'may': 5, 'jun': 6, 'jul': 7, 'aug': 8,
        'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12
    }
    df['month_numeric'] = df['month'].map(month_mapping)
    df['month_sin'] = np.sin(2 * np.pi * df['month_numeric'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month_numeric'] / 12)
    
    # ===== 4. ローン関連の特徴量 =====
    df['total_loans'] = (df['housing'] == 'yes').astype(int) + (df['loan'] == 'yes').astype(int)
    df['has_any_loan'] = (df['total_loans'] > 0).astype(int)
    
    # ===== 5. カテゴリカル特徴量の準備 =====
    binary_cols = ['default', 'housing', 'loan']
    for col in binary_cols:
        df[col] = df[col].map({'yes': 1, 'no': 0})
    
    # ワンホットエンコーディング対象のカテゴリカル変数
    categorical_cols = ['job', 'marital', 'education', 'contact', 'poutcome', 'age_group']
    
    # ===== 6. 相互作用特徴量 =====
    df['job_education'] = df['job'].astype(str) + '_' + df['education'].astype(str)
    df['contact_month'] = df['contact'].astype(str) + '_' + df['month'].astype(str)
    
    interaction_cols = ['job_education', 'contact_month']
    categorical_cols.extend(interaction_cols)
    
    # monthは既に周期性エンコーディングしたので削除
    df = df.drop(columns=['month', 'month_numeric'])
    
    return df, categorical_cols

# 特徴量エンジニアリングを適用
train_processed, categorical_cols = feature_engineering(train_balanced, is_train=True)
test_processed, _ = feature_engineering(test_df, is_train=False)

print("特徴量エンジニアリング完了")
print(f"Train shape: {train_processed.shape}")

In [ ]:
# ワンホットエンコーディング実行
train_encoded = pd.get_dummies(train_processed, columns=categorical_cols, drop_first=True)
test_encoded = pd.get_dummies(test_processed, columns=categorical_cols, drop_first=True)

# LightGBMのためにすべてのカラム名から特殊文字を除去
def sanitize_column_names(df):
    """LightGBM用にカラム名から特殊JSON文字を除去"""
    df.columns = df.columns.str.replace(r'[\(\)\[\]\{\}\:\"\,\.\s]', '_', regex=True)
    # 連続するアンダースコアを1つに
    df.columns = df.columns.str.replace(r'_+', '_', regex=True)
    # 先頭と末尾のアンダースコアを除去
    df.columns = df.columns.str.strip('_')
    return df

train_encoded = sanitize_column_names(train_encoded)
test_encoded = sanitize_column_names(test_encoded)

# 訓練データとテストデータのカラムを揃える
missing_cols = set(train_encoded.columns) - set(test_encoded.columns)
for col in missing_cols:
    if col != 'y':
        test_encoded[col] = 0

extra_cols = set(test_encoded.columns) - set(train_encoded.columns)
test_encoded = test_encoded.drop(columns=list(extra_cols))

# カラムの順序を揃える
test_encoded = test_encoded[train_encoded.drop(columns=['y']).columns]

print(f"ワンホットエンコーディング後のTrain shape: {train_encoded.shape}")
print(f"ワンホットエンコーディング後のTest shape: {test_encoded.shape}")

In [ ]:
# ターゲットと特徴量の分離
y = train_encoded['y']
X = train_encoded.drop(columns=['id', 'y'])
X_test = test_encoded.drop(columns=['id'])

# Train/Valid分割（バランスを保つためstratify使用）
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
)

print(f"特徴量数: {X.shape[1]}")
print(f"Train set: {X_train.shape}")
print(f"Valid set: {X_valid.shape}")
print(f"\nTrain set 正例率: {y_train.mean():.4f}")
print(f"Valid set 正例率: {y_valid.mean():.4f}")

## LightGBMのハイパーパラメータ最適化

In [ ]:
def objective_lgb(trial):
    """
    LightGBMのハイパーパラメータ最適化
    """
    params = {
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 20, 150),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "min_child_samples": trial.suggest_int("min_child_samples", 10, 100),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        "min_split_gain": trial.suggest_float("min_split_gain", 0.0, 1.0),
        "n_estimators": 2000,
        "objective": "binary",
        "metric": "auc",
        "verbosity": -1,
        "random_state": RANDOM_STATE,
    }
    
    model = lgb.LGBMClassifier(**params)
    model.fit(
        X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        callbacks=[lgb.early_stopping(100, verbose=False)]
    )
    
    preds = model.predict_proba(X_valid)[:, 1]
    auc = roc_auc_score(y_valid, preds)
    return auc

# 最適化実行
print("ハイパーパラメータ最適化を開始します...")
study_lgb = optuna.create_study(direction="maximize", study_name="lgbm_balanced")
study_lgb.optimize(objective_lgb, n_trials=50, show_progress_bar=True)

print(f"\nBest AUC: {study_lgb.best_value:.5f}")
print(f"\nBest params:")
for key, value in study_lgb.best_params.items():
    print(f"  {key}: {value}")

## 最適パラメータで学習

In [ ]:
# 最適パラメータで学習
best_params = study_lgb.best_params.copy()
best_params.update({
    "n_estimators": 2000,
    "objective": "binary",
    "metric": "auc",
    "verbosity": -1,
    "random_state": RANDOM_STATE,
})

model_lgb = lgb.LGBMClassifier(**best_params)
model_lgb.fit(
    X_train, y_train,
    eval_set=[(X_valid, y_valid)],
    callbacks=[lgb.early_stopping(100, verbose=True)]
)

# 検証データで評価
pred_proba = model_lgb.predict_proba(X_valid)[:, 1]
auc = roc_auc_score(y_valid, pred_proba)

print(f"\nValidation AUC: {auc:.5f}")

## 最適閾値の探索

In [ ]:
# 最適な閾値を探索
best_threshold = 0.5
best_f1 = 0

for threshold in np.arange(0.3, 0.8, 0.01):
    pred_binary = (pred_proba > threshold).astype(int)
    f1 = f1_score(y_valid, pred_binary)
    if f1 > best_f1:
        best_f1 = f1
        best_threshold = threshold

print(f"最適閾値: {best_threshold:.3f}")
print(f"最適F1スコア: {best_f1:.5f}")

# 最適閾値での評価
pred_binary = (pred_proba > best_threshold).astype(int)
print(f"\n=== 最適閾値での評価 ===")
print(f"Accuracy: {accuracy_score(y_valid, pred_binary):.5f}")
print(f"\nConfusion Matrix:")
print(confusion_matrix(y_valid, pred_binary))
print(f"\nClassification Report:")
print(classification_report(y_valid, pred_binary))

## 特徴量重要度

In [ ]:
# 特徴量重要度の可視化
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': model_lgb.feature_importances_
}).sort_values('importance', ascending=False)

plt.figure(figsize=(10, 8))
plt.barh(feature_importance['feature'][:20], feature_importance['importance'][:20])
plt.xlabel('Importance')
plt.title('Top 20 Feature Importances')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

print("\nTop 20 重要な特徴量:")
print(feature_importance.head(20))

## テストデータ予測と提出ファイル作成

In [ ]:
# テストデータで予測
test_pred_proba = model_lgb.predict_proba(X_test)[:, 1]
test_pred_binary = (test_pred_proba > best_threshold).astype(int)

# 提出ファイル作成
submission = pd.DataFrame({
    'id': test_df['id'],
    'y': test_pred_binary
})

submission.to_csv('/home/user/bank/data/balanced_dataset_submission.csv', index=False, header=False)

print("提出ファイルを作成しました: balanced_dataset_submission.csv")
print(f"\n予測分布:")
print(submission['y'].value_counts())
print(f"\nPositive予測率: {submission['y'].mean():.4f}")

In [ ]:
# 確率値も保存（閾値調整用）
submission_proba = pd.DataFrame({
    'id': test_df['id'],
    'y_proba': test_pred_proba,
    'y_pred': test_pred_binary
})

submission_proba.to_csv('/home/user/bank/data/balanced_dataset_submission_with_proba.csv', index=False)
print("確率値付き提出ファイルも作成しました: balanced_dataset_submission_with_proba.csv")